In [4]:
pip install opencv-python

     |████████████████████████████████| 52.1MB 7.4MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [29]:
# Import libraries
from collections import defaultdict
import cv2
from glob import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
from tensorflow.keras import models
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from random import choice, sample
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [25]:
# File paths
train_file_path = "/Users/florencialeoni/code/its_all_in_the_family/data/train/"
train_relationships_path = "/Users/florencialeoni/code/its_all_in_the_family/csv_files/train_relationships.csv"
validation_path = "F09"

In [43]:
# Functions by Youness Mansar https://github.com/CVxTz/kinship_prediction
all_images = glob(train_file_path + "*/*/*.jpg")

train_images = [x for x in all_images if validation_path not in x]
val_images = [x for x in all_images if validation_path in x]

train_person_to_images_map = defaultdict(list)

ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

val_person_to_images_map = defaultdict(list)

for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

relationships = pd.read_csv(train_relationships_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

train = [x for x in relationships if validation_path not in x[0]]
val = [x for x in relationships if validation_path in x[0]]


def read_img(path):
    img = preprocessing.image.load_img(path, target_size=(160, 160))
    img = preprocessing.image.img_to_array(img)
#     img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img


def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])

        yield [X1, X2], labels

In [44]:
img = read_img('/Users/florencialeoni/code/its_all_in_the_family/data/train/F0009/MID1/P10569_face2.jpg')

In [45]:
img.shape

(160, 160, 3)

In [6]:
# Import FaceNet model and FaceNet weights
facenet_model = models.load_model("/Users/florencialeoni/code/its_all_in_the_family/facenet/facenet_keras.h5")
facenet_model.load_weights("/Users/florencialeoni/code/its_all_in_the_family/facenet/facenet_keras_weights.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# Model Summary 
facenet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
__________________________________________________________________________________________________
Conv2d_2a_

In [12]:
# Define Model Architecture 
def siamese_model():
    left_image = layers.Input(shape = (160, 160, 3))
    right_image = layers.Input(shape = (160, 160, 3))
    
    model = models.Sequential()
    model.add(facenet_model)
    model.add(layers.Dense(128, activation = "relu"))
    facenet_model.trainable = False
    
    x1 = model(left_image)
    x2 = model(right_image)
    
    L2_normalized_layer_1 = layers.Lambda(lambda x: K.l2_normalize(x, axis = 1))
    X1_normal = L2_normalized_layer_1(x1)
    X2_normal = L2_normalized_layer_1(x2)

    L1_layer = layers.Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([X1_normal, X2_normal])
    
    prediction = layers.Dense(1, activation = "sigmoid")(L1_distance)
    
    siamese_net = models.Model(inputs = [left_image, right_image], outputs = prediction)

    siamese_net.compile(loss = "binary_crossentropy", metrics = ["acc"], optimizer = optimizers.Adam(0.00001))
    
    return siamese_net

In [ ]:
kinship_model = siamese_model()
kinship_model.fit_generator(gen(train, train_person_to_images_map, batch_size = 16),
                    validation_data = gen(val, val_person_to_images_map, batch_size = 16), epochs = 100, verbose = 2,
                steps_per_epoch = 200, validation_steps = 100)

Epoch 1/100
100/100 [==============================] - 107s 1s/step - loss: 0.6971 - acc: 0.4881
 - 317s - loss: 0.6972 - acc: 0.4894 - val_loss: 0.6971 - val_acc: 0.4881
Epoch 2/100
100/100 [==============================] - 102s 1s/step - loss: 0.6957 - acc: 0.5006
 - 304s - loss: 0.6966 - acc: 0.4966 - val_loss: 0.6957 - val_acc: 0.5006
Epoch 3/100
100/100 [==============================] - 90s 904ms/step - loss: 0.6957 - acc: 0.5131
 - 279s - loss: 0.6961 - acc: 0.5003 - val_loss: 0.6957 - val_acc: 0.5131
Epoch 4/100
100/100 [==============================] - 90s 901ms/step - loss: 0.6911 - acc: 0.5394
 - 276s - loss: 0.6940 - acc: 0.5034 - val_loss: 0.6911 - val_acc: 0.5394
Epoch 5/100
100/100 [==============================] - 90s 896ms/step - loss: 0.6924 - acc: 0.5294
 - 277s - loss: 0.6941 - acc: 0.5075 - val_loss: 0.6924 - val_acc: 0.5294
Epoch 6/100
100/100 [==============================] - 91s 913ms/step - loss: 0.6946 - acc: 0.5106
 - 276s - loss: 0.6944 - acc: 0.5181 - v